<a href="https://colab.research.google.com/github/Tirthankar4/Encryptix/blob/main/Encryptix_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'genre-classification-dataset-imdb:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1417162%2F2347441%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240807%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240807T081932Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4511398d19afe552b85813f38f506d4db8b6e6c0f7643812bf5346691562263c0ce4c8adbf0ba53952b388eb80baaef806dcba853117068f798a97d0cac7ba1dcebd04eebdf54732b7a3c80d2cb112707177dcfec604262eacf843d036f58648f182f65d3faedd18e70ff86645fe2b4d3e23d33737c588555b296cd479b98b8da087ebe0d6a06a5d591c059637efff638d7aec6dbd41754d725c4fc48f3ac6227d96f45a0136071fe0b0f5bbc3554690ea4ab57afdf7a1704f723584457ec71f44a798d8b0403c471233f06e6f9c27447d425328506407394ba2451fc8153760f06397783c22ef9550ed6ae360f1f2b728b0996c4451c1aaf694ad9f2fef2741'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 43729690 bytes downloaded
Downloaded and uncompressed: genre-classification-dataset-imdb
Data source import complete.


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import optuna
from sklearn.metrics import f1_score, classification_report

In [5]:
train_path = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt"
train_data = pd.read_csv(train_path, sep=':::', names=['Title', 'Genre', 'Description'], engine = 'python')

test_path = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt"
test_data = pd.read_csv(test_path, sep=':::', names=['Id', 'Title', 'Description'], engine = 'python')

test_soln_path = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt"
test_data_soln = pd.read_csv(test_soln_path, sep=':::', names=['Title', 'Genre', 'Description'], engine = 'python')

In [8]:
stemmer = LancasterStemmer()
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'pic.\S+', '', text)
    text = re.sub(r"[^a-zA-Z+']", ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text + ' ')
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    text = " ".join([i for i in words if i not in stopwords and len(i) > 2])
    text = re.sub("\s[\s]+", " ", text).strip()
    return text

train_data['Text_cleaning'] = train_data['Description'].apply(clean_text)
test_data['Text_cleaning'] = test_data['Description'].apply(clean_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
tfidf_vectorizer = TfidfVectorizer()

X = tfidf_vectorizer.fit_transform(train_data['Text_cleaning'])

X_test = tfidf_vectorizer.transform(test_data['Text_cleaning'])

In [10]:
y = train_data['Genre']
y_test = test_data_soln['Genre']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=69)

In [22]:
'''def objective(trial):
    param = {
        'C': trial.suggest_float('C', 0.0, 10.0),
        'tol': trial.suggest_float('tol', 1e-3, 1e-2, log = True),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sag']),
        'max_iter': trial.suggest_int('max_iter', 100, 500)
        }

    log_model = LogisticRegression(**param)
    log_model.fit(X_train, y_train)
    log_preds = log_model.predict(X_val)
    score = f1_score(y_val, log_preds, average = "macro")
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

Training on LogisticRegression is taking very much so hyperparameters cannot be optimized in a reasonable amount of time'''

'def objective(trial):\n    param = {\n        \'C\': trial.suggest_float(\'C\', 0.0, 10.0),\n        \'tol\': trial.suggest_float(\'tol\', 1e-3, 1e-2, log = True),\n        \'solver\': trial.suggest_categorical(\'solver\', [\'lbfgs\', \'sag\']),\n        \'max_iter\': trial.suggest_int(\'max_iter\', 100, 500)\n        }\n\n    log_model = LogisticRegression(**param)\n    log_model.fit(X_train, y_train)\n    log_preds = log_model.predict(X_val)\n    score = f1_score(y_val, log_preds, average = "macro")\n    return score\n\nstudy = optuna.create_study(direction=\'maximize\')\nstudy.optimize(objective, n_trials=10)\n\nTraining on LogisticRegression is taking very much so hyperparameters cannot be optimized in a reasonable amount of time'

In [19]:
log_model = LogisticRegression(solver = 'lbfgs', max_iter = 400, n_jobs = -1)
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)
print(classification_report(y_test, log_preds))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.54      0.24      0.33      1314
       adult        0.62      0.17      0.27       590
   adventure        0.73      0.12      0.21       775
   animation        0.44      0.01      0.02       498
   biography        0.00      0.00      0.00       264
      comedy        0.54      0.58      0.56      7446
       crime        0.32      0.01      0.02       505
 documentary        0.65      0.87      0.74     13096
       drama        0.53      0.80      0.64     13612
      family        0.56      0.06      0.10       783
     fantasy        0.67      0.01      0.01       322
   game-show        0.92      0.47      0.62       193
     history        0.00      0.00      0.00       243
      horror        0.67      0.54      0.60      2204
       music        0.71      0.35      0.47       731
     musical        1.00      0.00      0.01       276
     mystery        0.00      0.00      0.00       318
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
def objective(trial):
    param = {
        'alpha': trial.suggest_float('alpha', 0.0, 10.0)
        }

    nb_model = MultinomialNB(**param)
    nb_model.fit(X_train, y_train)
    nb_preds = nb_model.predict(X_val)
    score = f1_score(y_val, nb_preds, average = "macro")
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-08-07 08:50:06,730] A new study created in memory with name: no-name-019c0073-43db-4bb0-b9cd-da2347661104
[I 2024-08-07 08:50:07,551] Trial 0 finished with value: 0.04448224346588141 and parameters: {'alpha': 4.39592660730516}. Best is trial 0 with value: 0.04448224346588141.
[I 2024-08-07 08:50:08,304] Trial 1 finished with value: 0.04404506000886705 and parameters: {'alpha': 8.834820041394174}. Best is trial 0 with value: 0.04448224346588141.
[I 2024-08-07 08:50:09,420] Trial 2 finished with value: 0.04407190086653385 and parameters: {'alpha': 7.203573307659418}. Best is trial 0 with value: 0.04448224346588141.
[I 2024-08-07 08:50:09,875] Trial 3 finished with value: 0.044063920317365834 and parameters: {'alpha': 9.38967120650642}. Best is trial 0 with value: 0.04448224346588141.
[I 2024-08-07 08:50:10,451] Trial 4 finished with value: 0.044662992538159224 and parameters: {'alpha': 3.0809885861163187}. Best is trial 4 with value: 0.044662992538159224.
[I 2024-08-07 08:50:11,1

In [21]:
nb_model = MultinomialNB(**study.best_params)
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

print(classification_report(y_test, nb_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.62      0.09      0.16      1314
       adult        0.56      0.11      0.18       590
   adventure        0.71      0.08      0.15       775
   animation        0.50      0.00      0.00       498
   biography        0.00      0.00      0.00       264
      comedy        0.51      0.45      0.48      7446
       crime        0.67      0.00      0.01       505
 documentary        0.57      0.87      0.69     13096
       drama        0.46      0.81      0.59     13612
      family        0.56      0.01      0.01       783
     fantasy        0.00      0.00      0.00       322
   game-show        0.97      0.16      0.27       193
     history        0.00      0.00      0.00       243
      horror        0.74      0.34      0.47      2204
       music        0.85      0.12      0.20       731
     musical        1.00      0.01      0.03       276
     mystery        0.00      0.00      0.00       318
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
